In [1]:
import sys
sys.path.insert(1, '..')
from cocoa import covid19 as cc
from cocoa import world as cw
from cocoaplot import display as cd

In [2]:
p=cc.DataBase("jhu")
w=cw.WorldInfo()

JHU aka Johns Hopkins database selected ...
Available keys words are :  ['deaths', 'confirmed', 'recovered']


In [3]:
ww=w.getData()
len(w.getWorldCountries())
for i in w.getWorldCountries():
    if sum(ww["Country"]==i) != 1:
        print(i,sum(ww["Country"]==i))
ww.head()

Burma 0
Kosovo 0
Sao Tome and Principe 0
Taiwan* 0
West Bank and Gaza 0


,Country,Population,Area,Density
0,China,1439323776,9388211,153
1,India,1380004385,2973190,464
2,US,331002651,9147420,36
3,Indonesia,273523615,1811570,151
4,Pakistan,220892340,770880,287


In [4]:
import numpy as np
countries=w.getWorldCountries()
np.setdiff1d(p.get_locations(),countries)

array(['', 'Bolivia, Plurinational State of', 'Brunei Darussalam',
       'Congo', 'Congo, The Democratic Republic of the', "Côte d'Ivoire",
       'Holy See (Vatican City State)', 'Iran, Islamic Republic of',
       'Korea, Republic of', "Lao People's Democratic Republic",
       'Moldova, Republic of', 'Myanmar', 'Palestine, State of',
       'Russian Federation', 'Syrian Arab Republic',
       'Taiwan, Province of China', 'Tanzania, United Republic of',
       'United States', 'Venezuela, Bolivarian Republic of', 'Viet Nam'],
      dtype='<U37')

In [7]:
whichdata='deaths'
d=p.get_stats(type='Cumul',which=whichdata,location=countries,output='pandas')
d.head()

CocoaLookupError: No country match the key "Burma". Error.

In [8]:
from datetime import datetime, timedelta, time
import pandas as pd
    
lastdate=max(d["date"])
d0=d[d["date"]==lastdate].sort_values("country")

last1week=lastdate-timedelta(weeks=1)
d1=d[d["date"]==last1week].sort_values("country")

last2week=lastdate-timedelta(weeks=2)
d2=d[d["date"]==last2week].sort_values("country")

dateref=lastdate-timedelta(weeks=6)
dref=d[d["date"]==dateref].sort_values("country")

dd=dref.merge(d0.merge(d1.merge(d2,on="country",how="inner",suffixes=('1','2')),on="country",how="inner"),on="country",how="inner",suffixes=('0',''))

def setevol0(row):
    if row["cases1"]-row["cases0"]== 0:
        return -1
    else:
        return (row["cases"]-row["cases1"])*100./(row["cases1"]-row["cases0"])
dd = dd.assign(evol0=dd.apply(setevol0, axis=1))

def setevol1(row):
    if row["cases2"]-row["cases0"] == 0:
        return -1
    else:
        return (row["cases1"]-row["cases2"])*100./(row["cases2"]-row["cases0"])
dd = dd.assign(evol1=dd.apply(setevol1, axis=1))

dd["acc"]=dd["evol0"]-dd["evol1"]
dd.head()

NameError: name 'd' is not defined

In [ ]:
import geopandas as gpd

geojsondatafile = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json'

gdf=gpd.read_file(geojsondatafile)
gdf.columns = ['country_code', 'country', 'geometry']
gdf.head()

In [ ]:
merged = gdf.merge(dd, on = 'country', how = 'left')
merged=merged#[["country","geometry","evol0","evol1","acc","cases"]]
merged.head()

In [ ]:
merged[merged["cases"].isnull()]

In [ ]:
import json

#Read data to json
merged_json = json.loads(merged.to_json())

#Convert to str like object
json_data = json.dumps(merged_json)

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['RdYlGn'][10] # see https://docs.bokeh.org/en/latest/docs/reference/palettes.html

#Reverse color order so that dark blue is highest obesity.

hover = HoverTool(tooltips = [ ('Country','@country'),
                              ('Total '+whichdata, '@cases'),
                              ('Evolution last week in %','@evol0'),
                              ('Evolution previous week in %','@evol1') ] )

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = -50, high=50)
#Define custom tick labels for color bar.
#tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%'}#, '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')#, major_label_overrides = tick_labels)

#Create figure object.
f = figure(title = 'Acceleration (or deceleration) of weekly '+whichdata+' in % at '+str(lastdate), plot_height = 700 , plot_width = 950,tools = [hover])#, toolbar_location = None)
f.xgrid.grid_line_color = None
f.ygrid.grid_line_color = None

#Add patch renderer to figure. 
f.patches('xs','ys', source = geosource,fill_color = {'field' :'acc', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
f.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(f)

In [ ]:
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, HoverTool
from bokeh.palettes import brewer

#Input GeoJSON source that contains features for plotting.
geosource = GeoJSONDataSource(geojson = json_data)

#Define a sequential multi-hue color palette.
palette = brewer['YlOrRd'][8] # see https://docs.bokeh.org/en/latest/docs/reference/palettes.html
palette = palette[::-1]
#Reverse color order so that dark blue is highest obesity.d

hover = HoverTool(tooltips = [ ('Country','@country'),
                              ('Total deaths', '@cases'),
                              ('Evolution last week in %','@evol0'),
                              ('Evolution previous week in %','@evol1') ] )

#Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high=100)
#Define custom tick labels for color bar.
#tick_labels = {'0': '0%', '5': '5%', '10':'10%', '15':'15%', '20':'20%'}#, '25':'25%', '30':'30%','35':'35%', '40': '>40%'}

#Create color bar. 
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
border_line_color=None,location = (0,0), orientation = 'horizontal')#, major_label_overrides = tick_labels)

#Create figure object.
f = figure(title = whichdata+' evolution in % last week at '+str(lastdate), plot_height = 700 , plot_width = 950,tools = [hover])#, toolbar_location = None)
f.xgrid.grid_line_color = None
f.ygrid.grid_line_color = None

#Add patch renderer to figure. 
f.patches('xs','ys', source = geosource,fill_color = {'field' :'evol0', 'transform' : color_mapper},
          line_color = 'black', line_width = 0.25, fill_alpha = 1)

#Specify figure layout.
f.add_layout(color_bar, 'below')

#Display figure inline in Jupyter Notebook.
output_notebook()

#Display figure.
show(f)